<a href="https://colab.research.google.com/github/Akilesh1989/Running-TF-models-in-BigQuery/blob/main/IMDB_Reviews_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating a Tensorflow model


The following code creates a tensorflow model that predicts if a movie is good or bad. 1 stands for good and 0 stands for bad.

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

In [ ]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.models.Sequential([
  hub_layer,
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

In [ ]:
model.save('model_v1')

## Move the model to GCS

In [ ]:
from google.colab import auth

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
BUCKET = "akilesh-tensorflow-models"
!gsutil cp -r model_v1 gs://{BUCKET}/

## Convert train, test and validation datasets to CSV and move them to GCS

In [ ]:
def to_csv(data, filename):
  """Takes TF Dataset and writes it to a (local) CSV file. Make sure the dataset is not too large!"""
  import csv

  data_list = [{ 'text': text.decode('utf-8'), 'label': label } for text, label in data.as_numpy_iterator()]
  filename = '{}.csv'.format(filename)

  with open(filename, 'w') as f:
    writer = csv.DictWriter(f, data_list[0].keys())
    writer.writeheader()
    writer.writerows(data_list)

train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

to_csv(train_data, 'train')
to_csv(validation_data, 'validation')
to_csv(test_data, 'test')

In [ ]:
BUCKET = 'datasets-akilesh' # use your own bucket name here
!gsutil cp train.csv gs://{BUCKET}/
!gsutil cp validation.csv gs://{BUCKET}/
!gsutil cp test.csv gs://{BUCKET}/

## Create and load test data into the tables

### Authenticate COLAB to work with BQ

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
from google.cloud import bigquery

project_id = "adept-chemist-223208"
# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

dataset_id = "imdb"
table_name = "test"
table_id = f"{project_id}.{dataset_id}.{table_name}"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("text", "STRING"),
        bigquery.SchemaField("label", "INTEGER")
    ],
    skip_leading_rows=1,
    write_disposition="WRITE_TRUNCATE",
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
BUCKET = "datasets-akilesh"
file_name = "test.csv"
uri = f"gs://{BUCKET}/{file_name}"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

## Creating the model

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

MODEL_NAME = "movie_classification_model"

query = f"""
    CREATE OR REPLACE MODEL imdb.{MODEL_NAME}
    OPTIONS (MODEL_TYPE='TENSORFLOW',  MODEL_PATH="gs://akilesh-tensorflow-models/model_v1/*")

"""
query_job = client.query(query)  # Make an API request.

print("Loading the model into BigQuery:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)

## Evaluate the model

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

query = f"""
    WITH
 logits AS (
 SELECT
   *
 FROM
   ML.PREDICT( MODEL imdb.{MODEL_NAME},
     (
     SELECT
       text AS keras_layer_input,
       label
     FROM
       `imdb.test` ) ) ),
 predictions AS (
 SELECT
   CAST(1.0 / (1.0 + EXP(-dense_1)) > 0.5 AS INT64) AS pred,
   label
 FROM
   logits )
SELECT
 SUM(pred * label + (1 - pred)*(1 - label))/COUNT(*)*100 AS accuracy
FROM
 predictions
"""
query_job = client.query(query)  # Make an API request.

print("Evaluating the model:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)
    break


### Understanding the SQL code for prediction

In the first step we 

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

query = f"""SELECT
   *
 FROM
   ML.PREDICT( MODEL imdb.{MODEL_NAME},
     (
     SELECT
       text AS keras_layer_input,
       label
     FROM
       `imdb.test` ) )
"""
query_job = client.query(query)  # Make an API request.

print("Evaluating the model:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)
    break


In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

query = f"""WITH
  logits AS (
  SELECT
    *
  FROM
    ML.PREDICT( MODEL imdb.{MODEL_NAME},
      (
      SELECT
        text AS keras_layer_input,
        label
      FROM
        `imdb.test` ) ) ),
  predictions AS (
  SELECT
    CAST(1.0 / (1.0 + EXP(-dense_1)) > 0.5 AS INT64) AS pred,
    label
  FROM
    logits )
SELECT * FROM predictions
"""
query_job = client.query(query)  # Make an API request.

print("Evaluating the model:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)

## Making predictions

Imdb website: https://www.imdb.com/?ref_=nv_home

In [ ]:
import pandas as pd
from google.cloud import bigquery


sample_reviews = pd.DataFrame({
    'text': [
             'This was an awful movie',
             'Excellent movie',
             'Not too bad',
             'Good background score but terrible plot',
             'As a former Erasmus student I enjoyed this film very much. It was so realistic and funny. It really picked up the spirit that exists among Erasmus students. I hope, many other students will follow this experience, too. However, I wonder if this movie is all that interesting to watch for people with no international experience. But at least one of my friends who has never gone on Erasmus also enjoyed it very much. I give it 9 out of 10.',
             'As a film lover I found the movie to be very enjoyable. The screenplay could have been better but the overall experience was excellent.',
             'I have never ever seen a movie like this before. The acting was terrible and you call that CGI. Duh.'
             "I get why some people hate this . It's because of the political message and how some people think that you need get empathy for Arthur's madness. But come on that is not the point and it will never be. Enjoy this masterpiece because Joaquin Phoenix and Todd Phillips overdid themselves with this movie . The acting,music and cinematography are just amazing ! Please enjoy the movie without overthinking it."
             "I've lost count of the number of times I have seen this movie, but it is more than 20. It has to be one of the best movies ever made. It made me take notice Morgan Freeman and Tim Robbins like I had never noticed any actors before. I have from a very young age been a huge fan of anything Stephen King writes and had already read the short story that this movie is based on years prior to seeing this movie. Not everything Stephen King has written that gets turned into a movie comes out well, but this is as close to perfection as it gets and has everything you could ever want in a movie.Something that is outstanding is the fact that it has no real action, no special effects and no gimmicks. 99% of the movie is just men in a prison uniforms talking. Yet it absolutely hooks you almost from the beginning and has you glued to the screen to the end.For me what really makes this film one of the best is the message of eternal hope it conveys throughout. The never ever give up hope attitude of the main character so well conveyed by Tim Robbins. The ending is just spine tingling every time I see it, no matter how many times I have seen it.Brilliant, brilliant movie and a must see for everyone."
             ]
})
sample_reviews
sample_reviews.to_csv("sample.csv", index=False)


!gsutil cp sample.csv gs://{BUCKET}/


In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

dataset_id = "imdb"
table_name = "samples"
table_id = f"{project_id}.{dataset_id}.{table_name}"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("text", "STRING")
    ],
    skip_leading_rows=1,
    write_disposition="WRITE_TRUNCATE",
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
BUCKET = "datasets-akilesh"
file_name = "sample.csv"
uri = f"gs://{BUCKET}/{file_name}"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

query = f"""
    WITH

logistic_predictions AS (
SELECT
  *
FROM
  ML.PREDICT(MODEL imdb.{MODEL_NAME}, (
    SELECT
      text AS keras_layer_input,
      text
    FROM
      `imdb.{table_name}` ))),

predictions AS (
  SELECT
    text,
    CAST(1.0 / (1.0 + EXP(-dense_1)) > 0.5 AS INT64) AS pred
  FROM
    logistic_predictions)
SELECT * FROM predictions
"""
query_job = client.query(query)  # Make an API request.

print("Making predictions:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row[0], row[1])

References: https://medium.com/g-company/custom-model-on-bigquery-ml-45db14aa887a